## Data cleaning and preprocessing on the train.csv from kaggle https://www.kaggle.com/c/nfl-big-data-bowl-2020/data

In [4]:
#initial import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datetime
# from kaggle.competitions import nflrush
import tqdm
import re
from string import punctuation
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
import keras.backend as K
import tensorflow as tf

sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [15,10]

Using TensorFlow backend.


In [ ]:
# !pip install kaggle
# from kaggle.competitions import nflrush

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datetime
import tqdm
import re
from string import punctuation

In [6]:
train0 = pd.read_csv('train.csv', dtype={'WindSpeed': 'object'})
train = train0.copy()

In [7]:
train.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


In [8]:
train['DefendersInTheBox_vs_Distance'] = train['DefendersInTheBox'] / train['Distance']

In [9]:
train['StadiumType'].value_counts()

Outdoor                   267696
Outdoors                   67474
Indoors                    40854
Dome                       17336
Indoor                     16148
Retractable Roof           15884
Open                        9614
Retr. Roof-Closed           7172
Retr. Roof - Closed         6446
Domed, closed               5918
Domed, open                 2684
Closed Dome                 2134
Domed                       1826
Dome, closed                1826
Oudoor                      1188
Indoor, Roof Closed         1056
Retr. Roof Closed           1056
Retr. Roof-Open              990
Bowl                         968
Outddors                     968
Heinz Field                  902
Outdoor Retr Roof-Open       880
Retr. Roof - Open            880
Indoor, Open Roof            858
Outdor                       858
Ourdoor                      858
Outside                      814
Domed, Open                  770
Cloudy                       770
Name: StadiumType, dtype: int64

In [10]:
# Modify stadium types and get outdoor/indoor open/close info
def StadiumTypeAlter(i):
    if pd.isna(i):
        return np.nan
    i = i.lower()
    i = ''.join([c for c in i if c not in punctuation])
    i = re.sub(' +', ' ', i)
    i = i.strip()
    i = i.replace('outside', 'outdoor')
    i = i.replace('outdor', 'outdoor')
    i = i.replace('outddors', 'outdoor')
    i = i.replace('outdoors', 'outdoor')
    i = i.replace('oudoor', 'outdoor')
    i = i.replace('indoors', 'indoor')
    i = i.replace('ourdoor', 'outdoor')
    i = i.replace('retractable', 'rtr.')
    if 'outdoor' in i or 'open' in i:
        return 1
    if 'indoor' in i or 'closed' in i:
        return 0    
    return np.nan

In [11]:
train['StadiumType'] = train['StadiumType'].apply(StadiumTypeAlter)

In [12]:
train['StadiumType'].head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: StadiumType, dtype: float64

In [13]:
train['Turf'].value_counts()

Grass                     166276
Natural Grass             107096
Field Turf                 47674
Artificial                 44484
FieldTurf                  33198
UBU Speed Series-S5-M      32890
A-Turf Titan               17204
UBU Sports Speed S5-M       8558
FieldTurf360                8272
DD GrassMaster              8074
Twenty-Four/Seven Turf      8030
SISGrass                    7238
FieldTurf 360               6974
Natural grass               4796
Artifical                   3894
Natural                     1628
Field turf                   990
Naturall Grass               858
natural grass                814
grass                        814
Name: Turf, dtype: int64

In [14]:
# Modify Turf
Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 
        'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 
        'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 
        'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 
        'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 

train['Turf'] = train['Turf'].map(Turf)
train['Turf'] = train['Turf'] == 'Natural'

train['Turf'].head()

0    False
1    False
2    False
3    False
4    False
Name: Turf, dtype: bool

In [15]:
# Modify Possession Team
train[(train['PossessionTeam']!=train['HomeTeamAbbr']) & (train['PossessionTeam']!=train['VisitorTeamAbbr'])][['PossessionTeam', 'HomeTeamAbbr', 'VisitorTeamAbbr']]
diff_abbr = []
for x,y  in zip(sorted(train['HomeTeamAbbr'].unique()), sorted(train['PossessionTeam'].unique())):
    if x!=y:
        print(x + " " + y)

ARI ARZ
BAL BLT
CLE CLV
HOU HST


In [16]:
map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
for abb in train['PossessionTeam'].unique():
    map_abbr[abb] = abb
train['PossessionTeam'] = train['PossessionTeam'].map(map_abbr)
train['HomeTeamAbbr'] = train['HomeTeamAbbr'].map(map_abbr)
train['VisitorTeamAbbr'] = train['VisitorTeamAbbr'].map(map_abbr)
train['HomePossesion'] = train['PossessionTeam'] == train['HomeTeamAbbr']
train['Field_eq_Possession'] = train['FieldPosition'] == train['PossessionTeam']
train['HomeField'] = train['FieldPosition'] == train['HomeTeamAbbr']
train['PossessionTeam'].head()

0    NE
1    NE
2    NE
3    NE
4    NE
Name: PossessionTeam, dtype: object

In [17]:
# Check offense Formation,seems no need to modify
train['OffenseFormation'].value_counts()

SINGLEBACK    225434
SHOTGUN       150964
I_FORM        106062
PISTOL         13420
JUMBO          11462
WILDCAT         1782
EMPTY            506
ACE               22
Name: OffenseFormation, dtype: int64

In [18]:
# Game Clock: divide the Game Clock by 15 minutes to get normalized time left in the quarter
train['GameClock'].value_counts()

15:00:00    14476
02:00:00     5236
14:54:00     2156
14:55:00     1958
14:56:00     1276
            ...  
00:01:00      110
14:58:00       88
14:59:00       66
14:39:00       44
00:00:00       22
Name: GameClock, Length: 901, dtype: int64

In [19]:
def timeclock(i):
    i = i.split(':')
    out = int(i[0])*60 + int(i[1]) + int(i[2])/60
    return out
train['GameClock'] = train['GameClock'].apply(timeclock)

In [20]:
train['GameClock'].head()

0    854.0
1    854.0
2    854.0
3    854.0
4    854.0
Name: GameClock, dtype: float64

In [21]:
# Turn PlayerHeight into standardized numerical data (unit: inch)
train['PlayerHeight'] = train['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

Add BMI as an indicator of health status of players

In [22]:
# Create a new column showing the BMIs of players
train['PlayerBMI'] = 703*(train['PlayerWeight']/(train['PlayerHeight'])**2)

In [23]:
# Create a new column showing the durations of plays
train['TimeHandoff'].head()

0    2017-09-08T00:44:06.000Z
1    2017-09-08T00:44:06.000Z
2    2017-09-08T00:44:06.000Z
3    2017-09-08T00:44:06.000Z
4    2017-09-08T00:44:06.000Z
Name: TimeHandoff, dtype: object

In [24]:
train['TimeHandoff'] = train['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train['TimeSnap'] = train['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train['PlayDuration'] = train.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
train['PlayDuration'].head()
# Durations could be quite similar, but the data still worthy of exploratory analysis

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: PlayDuration, dtype: float64

In [25]:
# Similarly, we create a column for players' ages by processing players' birth dates (unit: year)
train['PlayerBirthDate'] = train['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
seconds_in_year = 60*60*24*365.25
train['PlayerAge'] = train.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
train['PlayerAge'].head()

0    28.692760
1    28.457305
2    28.629790
3    34.795430
4    30.061685
Name: PlayerAge, dtype: float64

In [26]:
# Standardize Wind Speed and Direction data
train['WindSpeed'].value_counts()

5                    53284
6                    41580
7                    39578
4                    34584
9                    31328
10                   29788
8                    29370
3                    26862
2                    24112
12                   23584
11                   17116
15                   13926
0                    13772
1                    12078
16                    9878
13                    8404
14                    6094
17                    3872
18                    1980
13 MPH                1804
23                    1166
E                     1144
SE                    1122
7 MPH                 1100
10-20                 1100
Calm                  1100
12-22                 1056
6 mph                 1034
20                    1012
12mph                  968
14-23                  968
24                     968
10MPH                  902
4 MPh                  902
10mph                  902
SSW                    836
15 gusts up to 25      836
2

In [27]:
train['WindSpeed'] = train['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
train['WindSpeed'] = train['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
train['WindSpeed'] = train['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)

def str_to_float(txt):
    try:
        return float(txt)
    except:
        return -1
train['WindSpeed'] = train['WindSpeed'].apply(str_to_float)

In [28]:
train['WindDirection'].value_counts()

NE                 30250
NW                 27236
SW                 25828
SE                 25784
WSW                24222
N                  23188
W                  22198
S                  21384
NNE                20394
South              20328
SSW                19910
WNW                19118
North              17182
NNW                14036
West               13618
SSE                13376
E                  12826
ENE                10802
ESE                 9878
East                7348
Northwest           4070
From SW             3872
Northeast           3652
NorthEast           3212
From S              3146
s                   2728
SouthWest           2134
Southeast           1936
Southwest           1804
W-NW                1804
West-Southwest      1386
8                   1144
1                   1122
North East          1100
East Southeast      1078
West Northwest      1056
Calm                1056
N-NE                1012
From W               990
From NNE             968


In [29]:
def clean_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = txt.replace('from', '')
    txt = txt.replace(' ', '')
    txt = txt.replace('north', 'n')
    txt = txt.replace('south', 's')
    txt = txt.replace('west', 'w')
    txt = txt.replace('east', 'e')
    return txt
train['WindDirection'] = train['WindDirection'].apply(clean_WindDirection)

In [30]:
def transform_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    
    if txt=='n':
        return 0
    if txt=='nne' or txt=='nen':
        return 1/8
    if txt=='ne':
        return 2/8
    if txt=='ene' or txt=='nee':
        return 3/8
    if txt=='e':
        return 4/8
    if txt=='ese' or txt=='see':
        return 5/8
    if txt=='se':
        return 6/8
    if txt=='ses' or txt=='sse':
        return 7/8
    if txt=='s':
        return 8/8
    if txt=='ssw' or txt=='sws':
        return 9/8
    if txt=='sw':
        return 10/8
    if txt=='sww' or txt=='wsw':
        return 11/8
    if txt=='w':
        return 12/8
    if txt=='wnw' or txt=='nww':
        return 13/8
    if txt=='nw':
        return 14/8
    if txt=='nwn' or txt=='nnw':
        return 15/8
    return np.nan
train['WindDirection'] = train['WindDirection'].apply(transform_WindDirection)

In [31]:
# PlayDirection, Team
train['PlayDirection'].value_counts()

left     256454
right    253308
Name: PlayDirection, dtype: int64

In [32]:
train['PlayDirection'] = train['PlayDirection'].apply(lambda x: x.strip() == 'right')
train['Team'] = train['Team'].apply(lambda x: x.strip()=='home')

In [33]:
# Standardize Weather
train['GameWeather'].unique()

array(['Clear and warm', 'Sun & clouds', 'Sunny', 'Controlled Climate',
       'Mostly Sunny', 'Clear', nan, 'Indoor', 'Mostly Cloudy',
       'Mostly Coudy', 'Partly sunny', 'Partly Cloudy', 'Cloudy',
       'Sunny, highs to upper 80s', 'Indoors', 'Light Rain', 'Showers',
       'Partly cloudy', 'Partly Sunny', '30% Chance of Rain',
       'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
       'Rain', 'Cloudy, fog started developing in 2nd quarter', 'Coudy',
       'Rain likely, temps in low 40s.', 'Cold', 'N/A (Indoors)',
       'Clear skies', 'cloudy', 'Fair', 'Mostly cloudy',
       'Cloudy, chance of rain', 'Heavy lake effect snow', 'Party Cloudy',
       'Cloudy, light snow accumulating 1-3"', 'Cloudy and cold', 'Snow',
       'Hazy', 'Scattered Showers', 'Cloudy and Cool', 'N/A Indoor',
       'Rain Chance 40%', 'Clear and sunny', 'Mostly sunny',
       'Sunny and warm', 'Partly clear', 'Cloudy, 50% change of rain',
       'Clear and Sunny', '

In [34]:
train['GameWeather'] = train['GameWeather'].str.lower()
indoor = "indoor"
train['GameWeather'] = train['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
train['GameWeather'] = train['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)

In [35]:
# To deal with the variety of weather descriptions, some text analysis could be helpful
from collections import Counter
weather_count = Counter()
for weather in train['GameWeather']:
    if pd.isna(weather):
        continue
    for word in weather.split():
        weather_count[word]+=1
        
weather_count.most_common()[:15]

[('cloudy', 193952),
 ('sunny', 127468),
 ('partly', 58256),
 ('clear', 55594),
 ('rain', 28952),
 ('indoor', 26950),
 ('controlled', 12540),
 ('climate', 12540),
 ('and', 10956),
 ('cloudy,', 4972),
 ('fair', 4972),
 ('snow', 4708),
 ('cold', 4510),
 ('of', 4026),
 ('light', 3608)]

In [36]:
# Then we could decide on the measurement of weather conditions
def weather(i):
    out = 1
    if pd.isna(i):
        return 0
    if 'partly' in i:
        out*=0.5
    if 'climate controlled' in i or 'indoor' in i:
        return out*3
    if 'sunny' in i or 'sun' in i:
        return out*2
    if 'clear' in i:
        return out
    if 'cloudy' in i:
        return -out
    if 'rain' in i or 'rainy' in i:
        return -2*out
    if 'snow' in i:
        return -3*out
    return 0

train['GameWeather'] = train['GameWeather'].apply(weather)

In [37]:
# Create a new column to show player types
train['Rusher'] = train['NflId'] == train['NflIdRusher']

In [38]:
# Anchoring offense moving left from {0,0} (modify X column)
train['X'] = train.apply(lambda row: row['X'] if row['PlayDirection'] else 120-row['X'], axis=1)
def new_orientation(a, play_direction):
    if play_direction == 0:
        new_angle = 360.0 - a
        if new_angle == 360.0:
            new_angle = 0.0
        return new_angle
    else:
        return a    
train['Orientation'] = train.apply(lambda row: new_orientation(row['Orientation'], row['PlayDirection']), axis=1)
train['Dir'] = train.apply(lambda row: new_orientation(row['Dir'], row['PlayDirection']), axis=1)

In [39]:
# At last we clean the YardsLeft columnn to get data of yards to the end zone
train['YardsLeft'] = train.apply(lambda row: 100-row['YardLine'] if row['HomeField'] else row['YardLine'], axis=1)
train['YardsLeft'] = train.apply(lambda row: row['YardsLeft'] if row['PlayDirection'] else 100-row['YardsLeft'], axis=1)
# Drop lines with obviously wrong data
train.drop(train.index[(train['YardsLeft']<train['Yards']) | (train['YardsLeft']-100>train['Yards'])], inplace=True)

In [40]:
train.to_csv("kaggle_train_processed.csv")

## Adding team stats into the dataset

In [41]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

In [42]:
# convert the abbreviation team name of the pbp file to the abbreviation team name used by the pro-football-reference website
def pbp_to_pro(abbr):
    corr_dict = {'ne':'nwe', 'no':'nor', 'lac':'sdg', 'sd':'sdg', 'la':'ram', 'sf':'sfo', 'tb':'tam', 'kc':'kan', 'gb':'gnb', 'ten':'oti','hou':'htx','bal':'rav','oak':'rai','ari':'crd','ind':'clt'}
    if abbr in corr_dict.keys():
        return corr_dict[abbr]
    return abbr

# convert the abbreviation team name of the train file to the abbreviation team name used by the pro-football-reference website
def train_to_pro(abbr):    
    train_name_dict = {'ARZ': 'crd', 'ARI': 'crd', 'ATL': 'atl', 'BLT': 'rav', 'BAL': 'rav', 'BUF': 'buf', 'CAR': 'car', 'CHI': 'chi', 'CIN': 'cin', 'CLV': 'cle', 'CLE': 'cle', 'DAL': 'dal', 'DEN': 'den', 'DET': 'det', 'GB': 'gnb', 'HST': 'htx', 'HOU': 'htx', 'IND': 'clt', 'JAX': 'jax', 'KC': 'kan', 'LAC': 'sdg', 'LA': 'ram', 'MIA': 'mia', 'MIN': 'min', 'NE': 'nwe', 'NO': 'nor', 'NYG': 'nyg', 'NYJ': 'nyj', 'OAK': 'rai', 'PHI': 'phi', 'PIT': 'pit', 'SF': 'sfo', 'SEA': 'sea', 'TB': 'tam', 'TEN': 'oti', 'WAS': 'was'}
    return train_name_dict[abbr]

# create a dict of full team name and abbreviation used by the pro-football-reference website
def get_team_name():
    team_name_dict = dict()
    url = 'https://www.pro-football-reference.com/teams/'
    response = requests.get(url)
    result_page = BeautifulSoup(response.content,'lxml')
    team_table = result_page.find("table", {"id": 'teams_active'}).tbody
    team_rows = team_table.find_all('tr')
    for row in team_rows:
        team = row.find('th', {'class': 'left'})
        if team is not None:
            team_name_dict[team.text] = team.find('a').get('href')[7:10]
    team_name_dict['San Diego Chargers'] = 'sdg'
    return team_name_dict

# input a full team name, return the abbreviation used by the pro-football-reference website 
def get_team_abbr(team_list):
    team_name_dict = get_team_name()
    result = []
    for team in team_list:
        result.append(team_name_dict[team])
    return result

# calculate the team rank given a list of teams and their stats in one category
def get_team_rank(stats):
    sorted_list = []
    index = 0
    diff = stats.max() - stats.min()
    for i in stats:
        percent = (i-stats.min())/diff
        if percent >= 0.75:
            sorted_list.append('A')
        elif percent >= 0.5:
            sorted_list.append('B')
        elif percent >= 0.25:
            sorted_list.append('C')
        else:
            sorted_list.append('D')
    return sorted_list

# reverse the rank(since sometimes higher number doesn't mean better)
def reverse_rank(stats):
    r_dict = {'A':'D', 'B':'C', 'C':'B', 'D':'A'}
    return r_dict[stats]

# Given a dataframe, calculate all team ranks on every columns of categories
def get_rank_df(df):
    df_copy = df.copy()
    columns = list(df_copy)
    for col in columns[3:-1]:
        df_copy[col] = get_team_rank(df[col])
    return df_copy


In [43]:
team_pass_2017 = pd.read_csv('2017_pass.csv')
team_pass_2017 = team_pass_2017[:-3]
team_rush_2017 = pd.read_csv('2017_rush.csv')
team_rush_2017 = team_rush_2017[:-3]
team_passdef_2017 = pd.read_csv('2017_passdef.csv')
team_passdef_2017 = team_passdef_2017[:-3]
team_rushdef_2017 = pd.read_csv('2017_rushdef.csv')
team_rushdef_2017 = team_rushdef_2017[:-3]

team_pass_2018 = pd.read_csv('2018_pass.csv')
team_pass_2018 = team_pass_2018[:-3]
team_rush_2018 = pd.read_csv('2018_rush.csv')
team_rush_2018 = team_rush_2018[:-3]
team_passdef_2018 = pd.read_csv('2018_passdef.csv')
team_passdef_2018 = team_passdef_2018[:-3]
team_rushdef_2018 = pd.read_csv('2018_rushdef.csv')
team_rushdef_2018 = team_rushdef_2018[:-3]

team_pass_2017['TmAbbr'] = get_team_abbr(team_pass_2017['Tm'])
team_rush_2017['TmAbbr'] = get_team_abbr(team_rush_2017['Tm'])
team_passdef_2017['TmAbbr'] = get_team_abbr(team_passdef_2017['Tm'])
team_rushdef_2017['TmAbbr'] = get_team_abbr(team_rushdef_2017['Tm'])

team_pass_2018['TmAbbr'] = get_team_abbr(team_pass_2018['Tm'])
team_rush_2018['TmAbbr'] = get_team_abbr(team_rush_2018['Tm'])
team_passdef_2018['TmAbbr'] = get_team_abbr(team_passdef_2018['Tm'])
team_rushdef_2018['TmAbbr'] = get_team_abbr(team_rushdef_2018['Tm'])

team_pass_rank_2017 = get_rank_df(team_pass_2017)
team_rush_rank_2017 = get_rank_df(team_rush_2017)
team_pass_rank_2017.set_index('TmAbbr', inplace=True)
team_rush_rank_2017.set_index('TmAbbr', inplace=True)
team_passdef_rank_2017 = get_rank_df(team_passdef_2017)
team_rushdef_rank_2017 = get_rank_df(team_rushdef_2017)
team_passdef_rank_2017.set_index('TmAbbr', inplace=True)
team_rushdef_rank_2017.set_index('TmAbbr', inplace=True)

team_pass_rank_2018 = get_rank_df(team_pass_2018)
team_rush_rank_2018 = get_rank_df(team_rush_2018)
team_pass_rank_2018.set_index('TmAbbr', inplace=True)
team_rush_rank_2018.set_index('TmAbbr', inplace=True)
team_passdef_rank_2018 = get_rank_df(team_passdef_2018)
team_rushdef_rank_2018 = get_rank_df(team_rushdef_2018)
team_passdef_rank_2018.set_index('TmAbbr', inplace=True)
team_rushdef_rank_2018.set_index('TmAbbr', inplace=True)

def add_team_stats(df, train=0):
    off_pass_cmp_list = []
    off_pass_att_list = []
    off_pass_cmprate_list = []
    off_pass_td_list = []
    off_intrate_list  = []
    off_sackrate_list = []
    off_pass_yds_list = []
    off_pass_yds_game_list = []
    off_pass_yds_atm_list = []
    off_qbr_list = []
    off_rush_att_list = []
    off_rush_yds_list = []
    off_rush_yds_game_list = []
    off_rush_yds_atm_list = []
    off_rush_td_list = []
    
    def_pass_cmp_list = []
    def_pass_att_list = []
    def_pass_cmprate_list = []
    def_pass_td_list = []
    def_intrate_list  = []
    def_sackrate_list = []
    def_pass_yds_list = []
    def_pass_yds_game_list = []
    def_pass_yds_atm_list = []
    def_rush_att_list = []
    def_rush_yds_list = []
    def_rush_yds_game_list = []
    def_rush_yds_atm_list = []
    def_rush_td_list = []
    
    for index, row in df.iterrows():
        if train == 0:
            all_list = [[team_pass_rank, team_rush_rank, team_passdef_rank, team_rushdef_rank]]
            off_team = pbp_to_pro(row[7].lower())
            def_team = pbp_to_pro(row[8].lower())
        else:
            all_list = [[team_pass_rank_2017, team_rush_rank_2017, team_passdef_rank_2017, team_rushdef_rank_2017], [team_pass_rank_2018, team_rush_rank_2018, team_passdef_rank_2018, team_rushdef_rank_2018]]
            season = row[14]
            if season == '2017':
                season = 0
            else:
                season = 1
            team1 = train_to_pro(row[38])
            team2 = train_to_pro(row[39])
            off_team = train_to_pro(row[18])
            if off_team == team1:
                def_team = team2
            else:
                def_team = team1
        off_pass_cmp_list.append(all_list[season][0]['Cmp'][off_team])
        off_pass_att_list.append(all_list[season][0]['Att'][off_team])
        off_pass_cmprate_list.append(all_list[season][0]['Cmp%'][off_team])
        off_pass_td_list.append(all_list[season][0]['TD'][off_team])
        off_intrate_list.append(reverse_rank(all_list[season][0]['Int%'][off_team]))
        off_sackrate_list.append(reverse_rank(all_list[season][0]['Sk%'][off_team]))
        off_pass_yds_list.append(all_list[season][0]['Yds'][off_team])
        off_pass_yds_game_list.append(all_list[season][0]['Y/G'][off_team])
        off_pass_yds_atm_list.append(all_list[season][0]['Y/A'][off_team])
        off_qbr_list.append(all_list[season][0]['Rate'][off_team])
        off_rush_att_list.append(all_list[season][1]['Att'][off_team])
        off_rush_yds_list.append(all_list[season][1]['Yds'][off_team])
        off_rush_yds_game_list.append(all_list[season][1]['Y/G'][off_team])
        off_rush_yds_atm_list.append(all_list[season][1]['Y/A'][off_team])
        off_rush_td_list.append(all_list[season][1]['TD'][off_team])
        
        def_pass_cmp_list.append(reverse_rank(all_list[season][2]['Cmp'][def_team]))
        def_pass_att_list.append(reverse_rank(all_list[season][2]['Att'][def_team]))
        def_pass_cmprate_list.append(reverse_rank(all_list[season][2]['Cmp%'][def_team]))
        def_pass_td_list.append(reverse_rank(all_list[season][2]['TD'][def_team]))
        def_intrate_list.append(all_list[season][2]['Int%'][def_team])
        def_sackrate_list.append(all_list[season][2]['Sk%'][def_team])
        def_pass_yds_list.append(reverse_rank(all_list[season][2]['Yds'][def_team]))
        def_pass_yds_game_list.append(reverse_rank(all_list[season][2]['Y/G'][def_team]))
        def_pass_yds_atm_list.append(reverse_rank(all_list[season][2]['Y/A'][def_team]))
        def_rush_att_list.append(reverse_rank(all_list[season][3]['Att'][def_team]))
        def_rush_yds_list.append(reverse_rank(all_list[season][3]['Yds'][def_team]))
        def_rush_yds_game_list.append(reverse_rank(all_list[season][3]['Y/G'][def_team]))
        def_rush_yds_atm_list.append(reverse_rank(all_list[season][3]['Y/A'][def_team]))
        def_rush_td_list.append(reverse_rank(all_list[season][3]['TD'][def_team]))
        
    df['Off Pass Cmp'] = off_pass_cmp_list
    df['Off Pass Att'] = off_pass_att_list
    df['Off Pass Cmp%'] = off_pass_cmprate_list
    df['Off Pass TD'] = off_pass_td_list
    df['Off Pass Int%'] = off_intrate_list
    df['Off Pass Sk%'] = off_sackrate_list
    df['Off Pass Yds'] = off_pass_yds_list
    df['Off Pass Y/G'] = off_pass_yds_game_list
    df['Off Pass Y/A'] = off_pass_yds_atm_list
    df['Off Pass QBR'] = off_qbr_list
    df['Off Rush Att'] = off_rush_att_list
    df['Off Rush Yds'] = off_rush_yds_list
    df['Off Rush Y/G'] = off_rush_yds_game_list
    df['Off Rush Y/A'] = off_rush_yds_atm_list
    df['Off Rush TD'] = off_rush_td_list

    df['Def Pass Cmp'] = def_pass_cmp_list
    df['Def Pass Att'] = def_pass_att_list
    df['Def Pass Cmp%'] = def_pass_cmprate_list
    df['Def Pass TD'] = def_pass_td_list
    df['Def Pass Int%'] = def_intrate_list
    df['Def Pass Sk%'] = def_sackrate_list
    df['Def Pass Yds'] = def_pass_yds_list
    df['Def Pass Y/G'] = def_pass_yds_game_list
    df['Def Pass Y/A'] = def_pass_yds_atm_list
    df['Def Rush Att'] = def_rush_att_list
    df['Def Rush Yds'] = def_rush_yds_list
    df['Def Rush Y/G'] = def_rush_yds_game_list
    df['Def Rush Y/A'] = def_rush_yds_atm_list
    df['Def Rush TD'] = def_rush_td_list
    
    return df

# add team rank stats to the train.csv
train = pd.read_csv('kaggle_train_processed.csv')
train_team = add_team_stats(train, 1)
train_team.to_csv('kaggle_train_processed_team.csv')

In [44]:
train = pd.read_csv('kaggle_train_processed_team.csv',index_col=0)
train.columns = ['Unnamed: 0.1', 'GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A', 'Dis',
       'Orientation', 'Dir', 'NflId', 'DisplayName', 'JerseyNumber', 'Season',
       'YardLine', 'Quarter', 'GameClock', 'PossessionTeam', 'Down',
       'Distance', 'FieldPosition', 'HomeScoreBeforePlay',
       'VisitorScoreBeforePlay', 'NflIdRusher', 'OffenseFormation',
       'OffensePersonnel', 'DefendersInTheBox', 'DefensePersonnel',
       'PlayDirection', 'TimeHandoff', 'TimeSnap', 'Yards', 'PlayerHeight',
       'PlayerWeight', 'PlayerBirthDate', 'PlayerCollegeName', 'Position',
       'HomeTeamAbbr', 'VisitorTeamAbbr', 'Week', 'Stadium', 'Location',
       'StadiumType', 'Turf', 'GameWeather', 'Temperature', 'Humidity',
       'WindSpeed', 'WindDirection', 'DefendersInTheBox_vs_Distance',
       'HomePossesion', 'Field_eq_Possession', 'HomeField', 'PlayerBMI',
       'PlayDuration', 'PlayerAge', 'Rusher', 'YardsLeft', 'Off_Pass_Cmp',
       'Off_Pass_Att', 'Off_Pass_CmpR', 'Off_Pass_TD', 'Off_Pass_IntR',
       'Off_Pass_SkR', 'Off_Pass_Yds', 'Off_Pass_Y_G', 'Off_Pass_Y_A',
       'Off_Pass_QBR', 'Off_Rush_Att', 'Off_Rush_Yds', 'Off_Rush_Y_G',
       'Off_Rush_Y_A', 'Off_Rush_TD', 'Def_Pass_Cmp', 'Def_Pass_Att',
       'Def_Pass_CmpR', 'Def_Pass_TD', 'Def_Pass_IntR', 'Def_Pass_SkR',
       'Def_Pass_Yds', 'Def_Pass_Y_G', 'Def_Pass_Y_A', 'Def_Rush_Att',
       'Def_Rush_Yds', 'Def_Rush_Y_G', 'Def_Rush_Y_A', 'Def_Rush_TD']
train.to_csv('kaggle_train_processed_team.csv')